## `lab08`—“The Secret Weapon that Won the War”

![](./img/lab09-header-bkgd.png)

❖ Objectives

-   Apply a brute-force attack to solve an encryption problem.

Way back in the `encode` lab, you built a tool to reproduce the encryption algorithm of the World War II-era Nazi Enigma machine.  Today, you will recapitulate some of the work required to solve such an encryption if the encoding offsets are unknown.

### Enigma:  A Refresher

Enigma uses a polyalphabetic cipher, in which each letter pressed on a mechanical keyboard would both be encoded and trigger a rotor to change position.  Since the rotor determines the alphabet (either the offset or a randomized substitution pattern), each letter press *changes the subsequent encoding alphabet*.  It's like changing the offset according to some rule every time you encoded a letter above.

To clarify this, first think of a pair of rotors, or wheels.  The inner (red) wheel represents the base alphabet (of the message), and the outer (blue) wheel represents the letter each inner letter maps to (the encoding).  An offset of one produces the following diagram:

![](./img/rotor-base.png)

A rotor cipher simply chains multiple wheels together, so that a change in one wheel produces an encoded letter *but also changes the position of the encoding rotor* for the next letter.  For instance, before encoding the letter `'A'` from the inner wheel, the rotor configuration is at left.  After encoding `'A'` (to `'B'`), the wheel advances and gives us the *new* configuration at right, in which `'A'` now maps to `'C'`.

![](./img/rotor-pair.png)

In order to think about a rotor cipher, you will have to accept a message, and for each letter in that message you will need to:

1.  Encode the letter.
2.  Advance the offset of the rotors by one.

-   We provide a function `rotor_cipher` which accepts a string `message` and a list `m` of the starting offsets for the $n$ rotors.  `rotor_cipher` should `return` the message transformed by successively apply the rotor cipher across the alphabet.

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

def mapper( letter,offset=1 ):
    '''
    Map a single-character string letter forward by its offset.
    '''
    return alphabet[ ( alphabet.index( letter ) + offset ) % 26 ]

def decode_n_rotors( message,m ):
    decoded = ''
    message = message.upper()
    
    for letter in message:
        if letter in alphabet:
            decoded += mapper( letter,sum( m ) )
            # Rotate all rotors back.
            m[ 0 ] = ( m[ 0 ] - 1 ) % 26
            for i in range( 1,len( m ) ):
                if m[ i - 1 ] % 26 == 0:
                    m[ i ] = ( m[ i ] - 1 ) % 26  # here as well
        else:
            decoded += letter
    
    # Finally, return the result.
    return encoded

def encode_n_rotors( message,m ):
    encoded = ''
    
    # Convert the message to upper-case.
    message = message.upper()
    
    # Loop over each letter of the message.
    for letter in message:
        # If the letter is in the alphabet, then:
        if letter in alphabet:
            # 1. encode the letter
            encoded += mapper( letter,sum( m ) )
            # 2. advance the offset of each rotor by n=1 (modulo 26)
            m[ 0 ] = ( m[ 0 ] + 1 ) % 26
            for i in range( 1,len( m ) ):
                if m[ i - 1 ] % 26 == 0:
                    m[ i ] = ( m[ i ] + 1 ) % 26  # here as well

        # Otherwise, just add the whitespace or punctuation or numeric character to the encoded string.
        else:
            encoded += letter
    
    # Finally, return the encoded message.
    return encoded

In [ ]:
# As an example, 'HELLO' maps to 'HFNOS' if it starts with an offset of 0 on one rotor:
#   H->H, E->F (1), L->N (2), L->O (3), O->S (4).
text = 'HELLO'
encode_n_rotors( text,[0,] )

In [ ]:
# As an example, 'HELLO' maps to 'HFNOS' if it starts with an offset of [1,1] on two rotors:
#   H->J (1+1), E->H (2+1), L->P (3+1), L->Q (4+1), O->U (5+1).
text = 'HELLO'
encode_n_rotors( text,[1,1] )

### A Brute-Force Solution

We will explore several potential solutions to this problem:

1.  A brute-force or exhaustive solution, which simply tries every single possible rotor configuration on an encrypted text.
2.  A one-letter frequency attack, based on the sort of calculations you did in the lab `language`.
3.  A three-letter frequency attack, based on the relative likelihood of a given three-letter combination in a language, like `ghi` or `nvy`.

First, the brute-force solution.  In this case, we make a guess about the number of rotors employed, try every combination, and visually inspect the results for the likely decryption.

For a single-rotor Enigma cipher, there is only one offset per letter, so only 26 possibilities need be tried for an exhaustive search.  For a two-rotor Enigma cipher, $26 \times 26 = 26^2 = 676$ cases must be tested.  In general, for $n$ rotors, $26^n$ cases must be searched.

We provide the function `decode_n_rotors( message,m )` which accepts an encrypted message `message` and a tuple or list of $n$ rotor offsets.

In [ ]:
def decode_n_rotors( message,m ):
    decoded = ''
    
    # Convert the message to upper-case.
    message = message.upper()
    
    # Reverse offsets for convenience.
    for i in range( len( m ) ):
        m[ i ] = -m[ i ]
    
    # Loop over each letter of the message.
    for letter in message:
        # If the letter is in the alphabet, then:
        if letter in alphabet:
            # 1. encode the letter
            decoded += mapper( letter,sum( m ) )
            # 2. decrement the offset of each rotor by n=1 (modulo 26)
            m[ 0 ] = ( m[ 0 ] - 1 ) % 26
            for i in range( 1,len( m ) ):
                if m[ i - 1 ] % 26 == 0:
                    m[ i ] = ( m[ i ] - 1 ) % 26  # here as well

        # Otherwise, just add the whitespace or punctuation or numeric character to the encoded string.
        else:
            decoded += letter

    # Finally, return the result.
    return decoded

In [ ]:
# Test [1,1] offset on two rotors.
test_text = """Mr. Fowler being a persevering man, as a good seaman should be, blockaded the house, and having met you
succeeded by certain arguments, metallic or otherwise, in convincing you that your interests were the same as his."""
code_text = """OU. JTCSMA LPUAU P FVJLYQAOGMG ODR, FY H OXYO EROBQE KAIPHA ZD, BMRGPGKMM DSQ UCJIV, SGX CWSGMG NGX DUB
ADMNQRRTT SQ VYMPXGM ASIXRKUBB, WPFNZAYT GK IODBPVITG, LR IVVESYOVBV OFM MBVP VMTR JPWIWLACC HQES IXV KTGZ WP FHS."""

print( encode_n_rotors( test_text,[ 1,1 ] ) )
assert encode_n_rotors( test_text,[ 1,1 ] ) == code_text

print( decode_n_rotors( code_text,[ 1,1 ] ) )
assert decode_n_rotors( code_text,[ 1,1 ] ) == test_text.upper()

In [ ]:
# Test [2,2,2] offset on three rotors.
test_text = """Mr. Fowler being a persevering man, as a good seaman should be, blockaded the house, and having met you
succeeded by certain arguments, metallic or otherwise, in convincing you that your interests were the same as his."""
code_text = """SY. NXGWQE PTYEY T JZNPCUESKQL SHV, JC L SBCS IVSFUI OEMTLE DH, FRVKTKOQQ HWU YGNMZ, WKB GAWKQK RLB HYF
EHQRUVVXX WU ZCQTBKQ EWMCVOYFF, ATJRDECX KO MSHFTZMXK, PW MZZIWCSZFZ SJQ QFZT ZQXV NTAMBPEGG LUIW MBZ OXKD AT JLW."""

print( encode_n_rotors( test_text,[ 2,2,2 ] ) )
assert encode_n_rotors( test_text,[ 2,2,2 ] ) == code_text

code_text = encode_n_rotors( test_text,[ 2,2,2 ] )
print( decode_n_rotors( code_text,[ 2,2,2 ] ) )
assert decode_n_rotors( code_text,[ 2,2,2 ] ) == test_text.upper()

Given these two tools, we can encrypt any message like the Enigma would.  We can also decrypt using candidate offsets in an attempt to solve an unknown message.

-   Consider the following message, which has been encoded by one rotor.  Write a loop which attempts all 26 possible rotor offsets and displays the result.  Use this to identify the correct offset.

In [ ]:
code_text = """KGH GVJV QDCSGWW"""

# Write a loop here to test all possible combinations for code_text with one rotor.

-   Consider the following message, which has been encoded by two rotors.  Write a nested loop which attempts all $26 \times 26 = 676$ possible rotor offsets ad displays the result.  Use this to identify the correct offset.

In [ ]:
code_text = """JBTDR HQQRG NQEO JAH IE"""

# Write a nested loop here to test all possible combinations for code_text with two rotors.

We are already in trouble.  We can solve it fairly quickly for 676 possibilities, but finding the solution is becoming a problem.  As we increase the number of rotors, this is a problem for brute-force solution.

One way out is to compare solutions to a dictionary of commonly used words, and flag possible matches.  Another is to analyze letter frequency in the result:  if it is close to the target language, then there may be a match.

### A Frequency-Based Solution

If we know that we are translating an English-language message, then we know the expected relative frequency of the letters.  Based on the principles outlined in `language`, we can automatically detect likely solutions even from many candidates.

Previously we used the language frequency to identify the most likely language.  For the Enigma code, we will instead use the language frequency to measure how close to English each sample is.  (Of course, it really would have been German, but we're working in English here.)

We'll need the ability to sort a dictionary of key-value pairs into two lists (of the keys and of the values), sorted by the values, in order to plot in order of value.  We can use this, for instance, to sort the standard English letter frequencies by order of frequency.  The blocks below carry out this sorting and plot it.  **None of these blocks requires modification.**

In [ ]:
# plotting boilerplate
import matplotlib as mpl               # import MatPlotLib
import matplotlib.pyplot as plt        # import PyPlot
%matplotlib inline
mpl.rcParams['figure.figsize'] = 15,3  # set the aspect ratio and size of the figure

# We'll use this function to plot the letters and frequencies for the next while.
# You don't need to interpret it yet, but you can examine it as much as you like.
def plot_freq(xs, ys, title=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    N = len(xs)

    ## necessary variables
    import numpy as np    # the Numerical Python package---you'll see this soon in lecture
    index = np.arange(N)  # x locations for bars
    width = 0.75          # bar width

    ## the bars
    rects1 = ax.bar(index, ys, width, color='blue')

    # axes and labels
    ax.set_title(title)
    ax.set_ylabel('Proportion')

    ax.set_xlim(-width,len(index)+width)
    ax.set_ylim(0,.20)
    xTickMarks = xs
    ax.set_xticks(index+width/2)
    xtickNames = ax.set_xticklabels(xTickMarks)
    plt.setp(xtickNames, fontsize=10)
    
    plt.show()

In [ ]:
# Convert a dictionary to two lists sorted by the value of the second.
import operator
def dict2sort(in_dict):
    keys = sorted(in_dict, key=in_dict.get)[::-1]
    values = []
    for key in keys:
        values.append(in_dict[key])
    return keys, values

In [ ]:
# Get standard English frequencies, sorted by order of frequency.
english_dict = {'A':0.0834,'B':0.0154,'C':0.0273,'D':0.0414,'E':0.126,'F':0.0203,'G':0.0192,'H':0.0611,'I':0.0671,'J':0.0023,'K':0.0087,
                'L':0.0424,'M':0.0253,'N':0.068,'O':0.077,'P':0.0166,'Q':0.0009,'R':0.0568,'S':0.0611,'T':0.0937,'U':0.0285,'V':0.0106,
                'W':0.0234,'X':0.002,'Y':0.0204,'Z':0.0006}
letters, freqs = dict2sort(english_dict)
plot_freq(letters, freqs, title='Expected Letter Frequency of English-Language Text')

We'll use our functions from `language` to aid in this.  Recall the function `calc_freq`, which returned a `dict` containing the relative frequency of letters in a sample of text.

In [ ]:
# define your function here
from string import whitespace, punctuation, digits
from string import ascii_uppercase as alphabet

def calc_freq(text):
    # Create an empty frequency dictionary letter_freq.
    letter_freq = {}
    
    # Make text upper-case.
    text = text.upper()
    
    # Loop over each letter of the alphabet:
    for letter in alphabet:
        letter_freq[letter] = text.count(letter)
    
    # Make a copy of text without non-alphabet characters.
    from string import whitespace, punctuation, digits
    for character in whitespace+punctuation+digits:
        text = text.replace(character, '')
    
    # Normalize the frequencies and put the results back into letter_freq.
    for key in letter_freq.keys():
        letter_freq[key] = letter_freq[key] / len(text)
    
    # Finally, return the dict letter_freq.
    return letter_freq

In [ ]:
code = """'LAY “XKJNTTBDOI” SATJNBD YON RV VXMXNFZDD CTLIKRF IB DSQ ETQC FNGWAOQ VHPUH ICVYMBCTAAG QJ S WYXEJYK ASG FEQIBTJLWQ OM
UZFBNZ IBYMS. BNWLTANP CRP EHPYUUM IA PEGR PBRHV NY VACOYEVPAO KAY XKJNTTBDOI SATJNBD, UG WH QCEINP BOTAMNB IFYF BX NPRVBT QEV
BIRBQSIHCWI HUTXDDLNYS UKEUMCKKQ NF BP LRYKNZJV GMEWPRCW HL V OCZL PT FSRVBBJLWQ IOGYRTEY, XKKOUUCEPJ VYMMSNMGSH, QEV LI AKOSH.
UJH JZTKIVOYFNZ EHFTEYHO FLVPNYII GYM, QYHQISG, JYW LUHA FL DBEK GFYL, IWN T TNJT SYGLYI PEC BOOSYYGITN XFYOSGI WGK YSLIGBIU
WVJGAUNXE MF WCLFDOCIC QFD LFCWY IBUKBZGF HTSYFBKPA. F FNPF UKTXATH DZ SVJT QE SVWJQKR NF UJH VKSICSZZF CU JYW VIHLRRZBMG QYRIMAC,
QGAQIYFFL, UIZ PM EOSVK XT UVN KYAGVTH. KZBM REIJ HNDNXHJ G KNFPXBDBUEL HZ ODB RGEPTB SK LBVMEUBBH EW S KYVH SYQIBDOI JDWZNDERR XD
KWKGN KC ANMQWWEGRL VDWNRFH. QTUHLYEKE SO NA GIKOUQCSZA, O CKDTXL DO ZMLPVVDFQK PN RDD PSRYDSE WVJ YC VRJVWIS JVEW LJ M ZPSYAGY.
(VHXL SUSKQK, 1937)\n'"""

code_dict = calc_freq(code)
code_letters, code_freqs = dict2sort(code_dict)
plot_freq(letters, freqs, title='Expected Letter Frequency of English-Language Text')
plot_freq(code_letters, code_freqs, title='Letter Frequency in Code Text')

The Caesar cipher is particularly weak, and only worked in an ancient world with low literacy and no prior cryptographic experience.  If you analyze the frequency of the letters, for instance, and guess that the most frequent letter in a long encoded text corresponds to E, the second most frequent to A, etc., then you can soon break Caesar ciphers even if you do not know the offset ahead of time.

However, the Enigma cipher is much more clever.  The occurrence of each letter is nearly uniform, leaving us no direct way to attempt to break this code by a frequency analysis!  Higher-order patterns must be identified and analyzed, which is precisely what the British Ultra project did before and during World War II.

Instead, we will use brute-force to solve the encodings, then use frequency analysis to recommend likely solutions to the user.

Consider the following encoded communiqué intercepted from a submarine:

In [ ]:
final_boss_encoded = """RBFRUU LH MPXJCRHG GYWOUO JDEMPY. SUGLA WCWOEDT.
NDWY KUMVI AAFWIYFF 0830A UE 9863, YLSQSG WAT
NBVMBPP GKTDKQ WYBNBCR, SQHII KPOQD VZBHH. Y RE
YIGHLUHNH VLJ KUMVI. MMECBUKWK ZVHIQ EOVTWJKU UK,
GTZQ BDHKZ-GIMPE-CZSU, HRVIL NXEC, HVGXRZDBNT PBL
MAVVLGFS URVPE"""

Your challenge is to solve this.  All you know is that it was encoded with a four-rotor Enigma device.

-   Compose a series of loops to solve the encoded message using three rotors.  Store the decrypted text in a dictionary `messages`.  The key should be the rotor setting (as a tuple) and the value should be the decrypted message.  This dictionary should have $26^3 = 17,576$ entries after you are done.  _This may take several minutes to run._  The `print` statement lets you monitor the process.

In [ ]:
messages = {}
for i in range( 26 ):
    for j in range( 26 ):
        print( i,j,end='\t' )
        for k in range( 26 ):
            text = ???
            messages[ ( i,j,k ) ] = ???

In [ ]:
# this is a testing cell; don't change it
assert len( messages ) == 17576

You have from `language` some tools which let you compare the similarity of letter frequency, such as `calc_match` and the reference letter frequency for English.

In [ ]:
def calc_match( L_text,L_ref ):
    '''
    Compute the difference of two dictionaries with the same keys.
    Args:
        L_text: The distribution of letter frequency of the analyzed text
        L_ref: The distribution of letter frequency of one language
    Returns:
        f: float, a caculated metric showing the difference between two dicts
    '''

    # Create an empty dictionary `L_diff`.
    L_diff = { }
    
    # Loop through the keys of the dictionaries (either by loading `alphabet` as above or by using `L_ref.keys()`).
    for letter in L_ref:
        # Calculate the absolute value of the difference between each dictionary value for each letter
        L_diff[ letter ] = abs( L_text[ letter ] - L_ref[ letter ] )
    
    # Next, loop through `L_diff` and sum all of the differences into the variable `f`.
    f = 0.0
    for letter in L_diff:
        f += L_diff[letter]
    
    # Finally, return the metric `f`.
    return f

In [ ]:
import requests
example_data = requests.get( 'https://raw.githubusercontent.com/UI-CS101/cs101-wiki/master/lab07/english' ).text
L_eng = { }
for line in example_data.split( '\n' ):
    try:
        letter,freq = line.split( ',' )
        L_eng[ letter ] = float( freq[ :-2 ] ) / 100
    except:
        pass

Try this on the first decryption attempt:

In [ ]:
first_attempt = messages[ ( 0,0,0 ) ]
print( first_attempt )
print( calc_match( calc_freq( first_attempt ),L_eng ) )

The actual frequency value by itself doesn't tell us much.  We just want to filter for the _best_ fits.

-   Compose a loop or series of loops to calculate the similarity $f$ using `calc_match` for every decryption attempt.  Store this result in a dictionary `fs`.  The key should be the rotor setting (as a tuple) and the value should be the fit $f$.  This dictionary should have $26^3 = 17,576$ entries after you are done.  _This may take several minutes to run._

In [ ]:
fs = { }
for key in messages:
    fs[ key ] = ???

-   Filter for the ten most likely keys:

In [ ]:
# no need to change this, it works already
dict2sort( fs )[ 0 ][ -10: ]

-   Output these candidate messages.  (There may be more than one good match due to the probabilistic approach we are taking.)

In [ ]:
# no need to change this, it works already
for key in dict2sort( fs )[ 0 ][ -10: ]:
    print( messages[ key ] )